In [13]:
## toolkit libraries to be installed

##data services
from qpds import repo_connector as data_repo
from qpds import universe
from qpds import calendar
from qpds import history
from qpds import backtest as backtest_data_fetch
from qpds.universe_repo import InIndex
from qpdcalc import environments


#rules and definition
from qidxrules.utils import rule_builder
from qidxdef import index

#backtest calculation
from qpdcalc import repo_connector as calc_repo
from qpdcalc import backtest as backtest_calc
from qpdcalc.async_task import create_backtest_job

#analytics
from qit import main


## external libraries
import os
import numpy as np
import pandas as pd
import datetime
import warnings
import time


warnings.filterwarnings('ignore')
pd.set_option("expand_frame_repr", False)

In [14]:

# %%
#Configure parameters needed for the get universe call
data_envt = 'PROD'
calc_envt = "PROD"
parent_symbol = "SXW1E"
ccy = 'EUR'
cal = 'STOXXCAL'
vendor_items = ['RBICS','Sustainalytics','ISS','EconsightMetaverse'] #'RBICS','Sustainalytics','ISS', RBICS_FOCUS
fields = ['TobaccoProdMaxRev','Total ESG Score','Environment Score']
sid_direct = True
calendar_name = 'STXSQ1'
start_at = datetime.date(2024, 1, 1)
end_at = datetime.date.today()

calc_input_type = 'file'


In [15]:
## bring your own data
vol_data = pd.read_csv(r"C:\Users\sradhakrishnan\OneDrive - ISS\Sriram\New Platform\iStudio\Vol_Data_3M_EUR.csv")

print(vol_data)

       effective_date    end_date   DJ_ID SEDOLNUMBER  infocode iso_ccy  no_obser       vol  vol_months
0          2021-06-21  2021-05-31  643491     6434915      2597     EUR        62  0.194734           3
1          2021-06-21  2021-05-31  JP611G     6485968      2629     EUR        61  0.234874           3
2          2021-06-21  2021-05-31  CN1GWC     6503525      2655     EUR        61  0.406553           3
3          2021-06-21  2021-05-31  CN02OP     6445467      2658     EUR        61  0.386131           3
4          2021-06-21  2021-05-31  CN02IJ     6478689      2661     EUR        61  0.262582           3
...               ...         ...     ...         ...       ...     ...       ...       ...         ...
250673     2024-12-23  2024-11-29  CN8WBE     BMDM0L7    358451     EUR        59  0.661528           3
250674     2024-12-23  2024-11-29  CNF5J2     BPDWSZ8    369594     EUR        59  0.899526           3
250675     2024-12-23  2024-11-29  CN8VXZ     BPDZ028    357937 

In [16]:


# %%
##indexDefinition : Create & Add rules to index
##################################
# Create Rules #

##selection filters
rule_1 = rule_builder.filter_simple(attribute='environment', operator='>=', value=0,special_inclusions=[str(pd.NA)] ,dataset_id='parent_universe')

rule_2 = rule_builder.filter_simple(attribute='TobaccoProdMaxRev', operator='>=', value=0,special_inclusions=[str(pd.NA),"Not Collected"] ,dataset_id='parent_universe')
rule_3 = rule_builder.filter_simple(attribute='Environment Score', operator='>=', value=50,special_exclusions=['<NA>'],dataset_id = 'parent_universe')

rule_4 = rule_builder.filter_simple(attribute='vol', operator='<=', value=0.30,special_exclusions=['<NA>'],dataset_id = 'parent_universe')

rule_6 = rule_builder.weight_by_attribute(attribute='ffmcap',dataset_id = 'parent_universe')
rule_7 = rule_builder.weight_factor(weight_attribute='weight', price_attribute="adjustedOpenPrice", scale_factor_billions=1)

##Index defintion 
idxdef = index.Index()
selection_stage = idxdef.add_stage(name='selection', calendar_name='STXSQ1',price_date="cutoff_date")
selection_stage.add_rules([rule_1,rule_2,rule_3,rule_4])
 
weighting_stage = idxdef.add_stage(name='weighting', calendar_name='STXWQ1',price_date="cutoff_date")
weighting_stage.add_rules([rule_6,rule_7])
 
idxdef.describe()
 


'{"index_parameter": {}, "stages": [{"name": "selection", "calendar_name": "STXSQ1", "price_date": "cutoff_date", "required_data": [], "rules": [{"rule": "filter_simple", "rule_type": 1, "rule_package": "qidxrules", "attribute": "environment", "operator": ">=", "value": 0, "special_inclusions": ["<NA>"], "dataset_id": "parent_universe"}, {"rule": "filter_simple", "rule_type": 1, "rule_package": "qidxrules", "attribute": "TobaccoProdMaxRev", "operator": ">=", "value": 0, "special_inclusions": ["<NA>", "Not Collected"], "dataset_id": "parent_universe"}, {"rule": "filter_simple", "rule_type": 1, "rule_package": "qidxrules", "attribute": "Environment Score", "operator": ">=", "value": 50, "special_exclusions": ["<NA>"], "dataset_id": "parent_universe"}, {"rule": "filter_simple", "rule_type": 1, "rule_package": "qidxrules", "attribute": "vol", "operator": "<=", "value": 0.3, "special_exclusions": ["<NA>"], "dataset_id": "parent_universe"}]}, {"name": "weighting", "calendar_name": "STXWQ1", 

In [17]:
# %%
#get calendar days
repo = data_repo.connect(data_envt)


some_days = calendar.get_calendar_days(repo,
                                        calendar_name,
                                        start_at,
                                        end_at)
#some_days = some_days[['cuttOffDay','effectiveDay']]
print(some_days)

date_pairs = list(zip(some_days['cuttOffDay'],some_days['effectiveDay']))
print(date_pairs)


environment: PROD
secrets_store: local
env: {'url_base': 'https://prod.stoxxistudio.com', 'sids_url_base': 'https://index-sid-vendor-api-46738465071.europe-west4.run.app', 'p12_content': b'0\x82\x12\x0f\x02\x01\x030\x82\x11\xc5\x06\t*\x86H\x86\xf7\r\x01\x07\x01\xa0\x82\x11\xb6\x04\x82\x11\xb20\x82\x11\xae0\x82\x07\xa2\x06\t*\x86H\x86\xf7\r\x01\x07\x06\xa0\x82\x07\x930\x82\x07\x8f\x02\x01\x000\x82\x07\x88\x06\t*\x86H\x86\xf7\r\x01\x07\x010W\x06\t*\x86H\x86\xf7\r\x01\x05\r0J0)\x06\t*\x86H\x86\xf7\r\x01\x05\x0c0\x1c\x04\x08\xeav\xc7\xd8\xae\x10V\xe7\x02\x02\x08\x000\x0c\x06\x08*\x86H\x86\xf7\r\x02\t\x05\x000\x1d\x06\t`\x86H\x01e\x03\x04\x01*\x04\x10\x99\xd9\'\x90 \xa3\x03b^!$\xe7v\x9f;\xa5\x80\x82\x07 #\xf7q\xf7\x01\xa2\x84\xcb4\x86m\xe2\xd0 H\x120\x81w\xbcXj\xd9Y\xf9\t\xbb#\x7f\t(\xa7\xe0@\x93\xa5\x00\xe1\x94\xea\xd0\xbb\xa4\xe4+iNN)\xca\x89\x10\x8c\xc1\xe5\x91\x98oS\xadQ=\xe4S\xc9\x93\xab\xa9({v\x12X!D\xe0\xe9h\xc7\xef\xa3\rF\x0fr\x8d\xce\x19+@\x82\x10*D>\xb6\xfcx\xe3f\x82\xd23\xae\x87\

In [18]:
# %%
## running historical index reviews - get universe call & running index deinition in the loop

index_def_result_all =pd.DataFrame() #creating a df to capture the consolidate the output of the review

for (securities_data_cutoff_date,composition_date) in date_pairs: # running for every review date in the backtest period
    
    result_initial = universe.get(repo, parent_symbol, securities_data_cutoff_date, composition_date,
                    cal, ccy,
                    fields,vendor_items,sid_direct=sid_direct)

    result_initial = pd.merge(result_initial,vol_data[vol_data['effective_date']== composition_date],left_on='stoxxid', right_on='DJ_ID',how='left')
    result_initial.to_csv(r"C:\Users\sradhakrishnan\OneDrive - ISS\Sriram\New Platform\iStudio\iDK_universe_output.csv",index=False)

    #run the selection and weighting rule defined for the index
    universe_df = {'parent_universe':result_initial}
    idxdef_result = idxdef.run(universe_df)['parent_universe']
    print(composition_date,len(idxdef_result.index),idxdef_result[idxdef_result['exclusion']== False].shape[0])

    #consolidate the backtested reviews
    index_def_result_all = pd.concat([index_def_result_all,idxdef_result])

#print backtested compositions -- only inclusions
print(index_def_result_all[index_def_result_all['exclusion']== False])

#save the backtested selection output including exclusions 
index_def_result_all.to_csv(r"C:\Users\sradhakrishnan\OneDrive - ISS\Sriram\New Platform\iStudio\iDK_selection_output.csv",index=False)


Success!
** Ready to fetch ['iss'] vendor data from SIDS REST API **
Success!
2024-03-18 1801 1200
Success!
** Ready to fetch ['iss'] vendor data from SIDS REST API **
Success!
2024-06-24 1801 1145
Success!
** Ready to fetch ['iss'] vendor data from SIDS REST API **
Success!
2024-09-23 1801 941
      compositionDate  securityDate inputCurrency stoxxid                 securityName          isin    sedol currency  icb5_l1  icb5_l2  ...       vol  vol_months exclusion  exclusion_reason  Step 1 environment exclusion Step 2 TobaccoProdMaxRev exclusion Step 3 Environment Score exclusion  Step 4 vol exclusion    weight weight_factor
1          20240318.0    20240229.0           EUR  005610     ASSOCIATED BRITISH FOODS  GB0006731235  0673123      GBP     45.0   4510.0  ...  0.159925         3.0     False                                           False                              False                              False                 False  0.000207  7.769967e+04
4          20240318.0    202

In [19]:
### no need to provide Index Symbol and ISIN while defining variants
### no need to pass the effective date additionally in the backtest data  
input_type = calc_input_type
env = environments.PROD
start_date = datetime.date(2024, 3, 18)
end_date = datetime.date.today()

backtest_data = index_def_result_all[index_def_result_all['exclusion']== False]
backtest_data['weight_factor']= backtest_data['weight_factor'].astype(int)
backtest_data['composition_date']= pd.to_datetime(backtest_data['compositionDate'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
backtest_data = backtest_data.rename(columns={'weight':'weights'})
backtest_data=backtest_data[['stoxxid','composition_date','weight_factor']].rename(columns={'weight_factor':'weightFactor'})



index_compositions = backtest_data.to_dict(orient='list')
print(index_compositions)

##################################
# index level methods #
##################################

# create index parameter
my_index_parameter = index.IndexParameter(index_name="idx sample 230",
                                            base_date= datetime.date(2024, 3, 18),
                                            base_value=1000,
                                            calculation_calendar="STOXXCAL",
                                            weight_adjustment_type="price weighted",
                                            rebalancing_type="None",
                                            parent_index="SXW1E",
                                            review_calendar="STXSQ1")
my_index_parameter.add_variants(index_symbol="DUMMY1",
                                isin="DD1234567890",
                                index_currency="EUR",
                                return_type="pr")
my_index_parameter.add_variants(index_symbol="DUMMY2",
                                isin="XX1234567890",
                                index_currency="USD",
                                return_type="nr")

# create index
my_index = index.Index()

# add index parameter to index
my_index.add_index_parameter(my_index_parameter)

print(my_index.describe())


data = create_backtest_job(env,
                               input_type,
                               my_index,
                               index_compositions,
                               start_date,
                               end_date)
    # Print the batch_id and task name
print({'batch_id': data['batch_id'],
        'message': 'Job started.'
        })

batchid = data['batch_id']

{'stoxxid': ['005610', '007937', '008118', '008320', '009417', '009616', '011517', '012715', '012905', '013670', '013849', '014084', '015400', '016419', '018270', '018390', '021623', '024249', '024282', '026256', '026527', '028758', '029440', '030964', '033086', '037178', '038112', '038470', '038580', '039600', '040054', '040520', '041610', '042186', '045449', '045644', '045796', '047640', '049065', '050450', '051152', '053315', '055768', '056540', '064623', '067760', '070995', '071887', '071921', '073087', '073303', '075069', '075478', '076764', '077940', '079087', '079851', '080216', '081410', '081660', '081827', '082201', '082699', '083470', '083860', '087061', '087541', '087823', '088470', '088869', '091321', '094660', '095298', '096050', '097404', '098111', '098952', '143406', '143451', '201164', '211850', '217052', '256612', '274642', '287864', '316383', '317731', '321430', '400169', '400609', '400702', '400930', '401093', '401140', '401190', '401225', '401632', '403197', '405097